In [14]:
import tensorflow as tf
import os
import pandas as pd

PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'sarah-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
SEQ_LEN = '50'

### Define the project, Bucket and Region and set them.

In [15]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'  # Tensorflow version
os.environ['SEQ_LEN'] = SEQ_LEN

In [16]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [17]:
df2 = pd.read_csv('data/CNN_W2P_model/train-1.csv', header = None)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.948333,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,...,0.658333,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667
1,0.942500,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,...,0.500000,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333
2,0.912333,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,...,1.085000,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500
3,0.500000,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,...,0.953333,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667
4,0.781000,0.654167,0.808333,0.724833,0.992833,0.668833,0.750000,0.830833,0.834500,0.816667,...,0.500000,0.846000,0.784667,0.751667,0.247333,1.001667,1.008333,0.790500,1.124667,0.492167


In [18]:
df2 = pd.read_csv('data/CNN_W2P_model/valid-1.csv', header = None) #, names = CSV_COLUMNS)
df2.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,...,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000,2638.000000
mean,0.827618,0.827851,0.827685,0.827717,0.827745,0.827840,0.827769,0.827687,0.827698,0.827717,...,0.829831,0.829818,0.829786,0.829823,0.829786,0.829683,0.829584,0.829590,0.829577,0.829446
std,0.217746,0.217454,0.217529,0.217534,0.217526,0.217463,0.217445,0.217395,0.217400,0.217403,...,0.215509,0.215509,0.215515,0.215481,0.215470,0.215397,0.215381,0.215381,0.215382,0.215333
min,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
25%,0.707250,0.707542,0.707250,0.707250,0.707250,0.707542,0.707542,0.707542,0.707542,0.707542,...,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625,0.710625
50%,0.842167,0.842833,0.842167,0.842833,0.842833,0.842833,0.842167,0.842167,0.842167,0.842833,...,0.844917,0.844833,0.844833,0.844833,0.844833,0.844833,0.844167,0.844833,0.844833,0.844167
75%,0.995667,0.995667,0.995667,0.995667,0.995667,0.995667,0.995667,0.995125,0.995125,0.995125,...,0.995000,0.995000,0.995000,0.995000,0.995000,0.994958,0.994833,0.994833,0.994833,0.994833
max,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,...,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500,1.417500


### Make sure the data is out in the bucket to train the hyperparameters on.

In [22]:
!gsutil cp -p /content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/valid-1.csv gs://sarah-bucket/CNN_W2P_model/valid-1.csv
!gsutil cp -p /content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/train-1.csv gs://sarah-bucket/CNN_W2P_model/train-1.csv



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying file:///content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/valid-1.csv [Content-Type=text/csv]...
/ [1 files][  1.5 MiB/  1.5 MiB]                                                
Operation completed over 1 objects/1.5 MiB.                                      
Copying file:///content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/train-1.csv [Content-Type=text/csv]...
/ [1 files][  2.3 MiB/  2.3 MiB]                                                
Operation completed over 1 objects/2.3 MiB.                                      


# Build a yaml

In [24]:
!pwd

/content/datalab/energy_forcasing/Sarah


In [26]:
%%writefile  hyperparam.yaml
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 50
    maxParallelTrials: 5
    hyperparameterMetricTag: rmse
    params:
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.01
      maxValue: 0.1
      scaleType: UNIT_LOG_SCALE
    - parameterName: train_batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LINEAR_SCALE

Overwriting hyperparam.yaml


In [27]:
%%bash

MODEL=dnn
OUTDIR=gs://${BUCKET}/CNN_W2P_model/${MODEL}
JOBNAME=CNN_W2P_model_${MODEL}$(date -u +%y%m%d_%H%M%S)

#gsutil -m rm -rf $OUTDIR
echo $JOBNAME

gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=trainer.task \
     --package-path=${PWD}/CNN_W2P_model/trainer \
     --job-dir=$OUTDIR \
     --scale-tier=PREMIUM_1 \
     --runtime-version=$TFVERSION \
     --config=hyperparam.yaml \
     -- \
     --train_data_path="gs://${BUCKET}/CNN_W2P_model/train-1.csv" \
     --eval_data_path="gs://${BUCKET}/CNN_W2P_model/valid-1.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=5000 --sequence_length=$SEQ_LEN --model=$MODEL
done

CNN_W2P_model_dnn180925_164529
jobId: CNN_W2P_model_dnn180925_164529
state: QUEUED


Job [CNN_W2P_model_dnn180925_164529] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe CNN_W2P_model_dnn180925_164529

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs CNN_W2P_model_dnn180925_164529
bash: line 22: syntax error near unexpected token `done'
bash: line 22: `done'


In [28]:
!gcloud ml-engine jobs describe CNN_W2P_model_dnn180925_164529

createTime: '2018-09-25T16:45:33Z'
etag: 5mQj5SRVH0Y=
jobId: CNN_W2P_model_dnn180925_164529
startTime: '2018-09-25T16:45:38Z'
state: RUNNING
trainingInput:
  args:
  - --train_data_path=gs://sarah-bucket/CNN_W2P_model/train-1.csv
  - --eval_data_path=gs://sarah-bucket/CNN_W2P_model/valid-1.csv
  - --output_dir=gs://sarah-bucket/CNN_W2P_model/dnn
  - --train_steps=5000
  - --sequence_length=50
  - --model=dnn
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: rmse
    maxParallelTrials: 5
    maxTrials: 50
    params:
    - maxValue: 0.1
      minValue: 0.01
      parameterName: learning_rate
      scaleType: UNIT_LOG_SCALE
      type: DOUBLE
    - maxValue: 512.0
      minValue: 8.0
      parameterName: train_batch_size
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
  jobDir: gs://sarah-bucket/CNN_W2P_model/dnn
  packageUris:
  - gs://sarah-bucket/CNN_W2P_model/dnn/packages/5beb883e1be7b4b4965a8d0ef5473f56b79d12d9f31e88f154fd0e06adebcb7b/trainer-0.0.0.tar.gz
  p

In [30]:
from google.datalab.ml import TensorBoard
TensorBoard().start('CNN_W2P_model'.format(BUCKET))

TensorBoard was started successfully with pid 6993. Click here to access it.

6993

In [29]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

NameError: name 'TensorBoard' is not defined

In [ ]:
## command line access of the TB
print('tensorboard --logdir=gs://{}/CNN_W2P_model --port=8083'.format(BUCKET))

## Deploy the model

### Convert the data into something that can be read into the model, so strings that are 49 long where the 50th price is perdicted.

In [65]:
%bash
gsutil ls gs://${BUCKET}/CNN_W2P_model/dnn/export/exporter/

gs://sarah-bucket/CNN_W2P_model/dnn/export/exporter/
gs://sarah-bucket/CNN_W2P_model/dnn/export/exporter/1537556449/


In [85]:
%bash
MODEL_NAME="CNN_W2P_model"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/CNN_W2P_model/dnn/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying CNN_W2P_model ml_on_gcp from gs://sarah-bucket/CNN_W2P_model/dnn/export/exporter/1537556449/ ... this will take a few minutes


Created ml engine model [projects/qwiklabs-gcp-aebfb78fe0f1b1d1/models/CNN_W2P_model].
Creating version (this might take a few minutes)......
............................................................................................done.


Make data to feed to model

In [88]:
def to_model_csv(filename, N, J):
  with open(filename, 'w') as ofp:
    N = df5.shape[0] - J
    for ii in xrange(0, N):
      seq = df5.price[(0+ii):(J+ii)]
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      #print(len(seq))
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass

to_model_csv('data/CNN_W2P_model/to_model.csv', N = 1000, J = 49) 


In [89]:
model_data = pd.read_csv('/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/to_model.csv', header = None)
model_data.shape

(6597, 49)

In [92]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME="CNN_W2P_model_dnn"
MODEL_VERSION="ml_on_gcp"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'price': prices
    },
  ]
}

In [93]:
## 55.11,62.0,46.79,44.0,44.0,44.86,60.89,58.69,36.49,57.69,51.2,48.79,48.35,42.2,46.37,62.5,62.5,54.0,52.54,62.18,48.5,55.35,52.0,40.97,42.6,64.1,61.59,48.9,37.4,45.93,37.3,65.48,62.5,65.03,57.07,23.35,54.56,28.02,39.84,51.0,42.46,47.02,37.1,38.0,38.0,60.99,50.01,19.52,39.98,47.96

This will take a few minutes.

In [94]:
#result = []
for y in range(1,10):#model_data.shape[0]):
    raw_prices = model_data.iloc[y]
    prices = [float(price) for price in raw_prices]
    #print prices
    data = {
      'instances': [
        {
          'price': prices
        },
      ]
    }
    #print(data)
    response = requests.post(api, json=data, headers=headers)
    #print y/model_data.shape[0]
    forecast = response.json()##['predictions'][0]['predicted'][0] ## (df5.price[(nn):(nn+J)])/60
    print (forecast)
    #result.append(response.content[32:-4])

{u'error': {u'status': u'NOT_FOUND', u'message': u'Field: name Error: The model resource: "CNN_W2P_model_dnn" was not found. Please create the Cloud ML model resource first by using \'gcloud ml-engine models create CNN_W2P_model_dnn\'.', u'code': 404, u'details': [{u'fieldViolations': [{u'field': u'name', u'description': u'The model resource: "CNN_W2P_model_dnn" was not found. Please create the Cloud ML model resource first by using \'gcloud ml-engine models create CNN_W2P_model_dnn\'.'}], u'@type': u'type.googleapis.com/google.rpc.BadRequest'}]}}
{u'error': {u'status': u'NOT_FOUND', u'message': u'Field: name Error: The model resource: "CNN_W2P_model_dnn" was not found. Please create the Cloud ML model resource first by using \'gcloud ml-engine models create CNN_W2P_model_dnn\'.', u'code': 404, u'details': [{u'fieldViolations': [{u'field': u'name', u'description': u'The model resource: "CNN_W2P_model_dnn" was not found. Please create the Cloud ML model resource first by using \'gcloud 

In [ ]:
print result

In [ ]:
!pwd
!ls

In [ ]:
%%writefile  hyperparam.yaml
trainingInput:
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 5
    maxParallelTrials: 1
    hyperparameterMetricTag: rmse
    params:
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.01
      maxValue: 0.1
      scaleType: UNIT_LOG_SCALE